In [1]:
from inVAE import FinVAE, NFinVAE

/home/icb/kemal.inecik/tools/apps/mamba/envs/cpa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import warnings
import os
import sys
import gc
import warnings

import anndata as ad
import scanpy as sc
import copy
import torch
from pathlib import Path
import networkx as nx
from sklearn.neighbors import kneighbors_graph
import numpy as np
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.stats

adata_file_path = os.path.join("/home/icb/kemal.inecik/lustre_workspace/tardis_data/processed", "dataset_complete_Suo.h5ad")
assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
adata = ad.read_h5ad(adata_file_path)
print(adata_file_path, flush=True)
print(adata, flush=True)

def create_random_mask(shape, ratio_true, seed=None):
    rng = np.random.default_rng(seed)
    random_floats = rng.random(shape)
    mask = random_floats < ratio_true
    return mask

/home/icb/kemal.inecik/lustre_workspace/tardis_data/processed/dataset_complete_Suo.h5ad
AnnData object with n_obs × n_vars = 841922 × 8192
    obs: 'sample_ID', 'organ', 'age', 'cell_type', 'sex', 'sex_inferred', 'concatenated_integration_covariates', 'integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse', 'n_genes'
    uns: 'rank_genes_groups'
    obsm: 'Unintegrated', 'X_pca', 'harmony'


In [3]:
print(f"CUDA: {torch.cuda.is_available()}", flush=True)

CUDA: False


In [4]:
adata_train_bool = create_random_mask(adata.shape[0], ratio_true=0.8, seed=0)
adata_train = adata[adata_train_bool].copy()
adata_valid = adata[~adata_train_bool].copy()

In [9]:
inv_covar_keys = {
    'cont': [],
    'cat': []
}

spur_covar_keys = {
    'cont': [],
    'cat': ['organ', 'integration_donor', 'integration_library_platform_coarse'] #set to the keys in the adata
}

model = FinVAE(
    adata = adata_train,
    layer = None,
    inv_covar_keys = inv_covar_keys,
    spur_covar_keys = spur_covar_keys,
    latent_dim_inv = 24, 
    latent_dim_spur = 8,
    decoder_dist = 'nb'
)

Layer is None, check if you want to specify the layer of adata!
Data loading done!


/home/icb/kemal.inecik/tools/apps/mamba/envs/cpa/lib/python3.9/site-packages/anndata/experimental/multi_files/_anncollection.py:210: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arrs.append(arr[idx])
/home/icb/kemal.inecik/tools/apps/mamba/envs/cpa/lib/python3.9/site-packages/anndata/experimental/multi_files/_anncollection.py:210: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arrs.append(arr[idx])


The covariates for the invariant prior are None. Defaulting to N(0,1) for that prior.


/home/icb/kemal.inecik/tools/apps/mamba/envs/cpa/lib/python3.9/site-packages/anndata/experimental/multi_files/_anncollection.py:210: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arrs.append(arr[idx])


In [10]:
model.train(n_epochs=500, lr_train=0.001, weight_decay=0.0001)

/home/icb/kemal.inecik/tools/apps/mamba/envs/cpa/lib/python3.9/site-packages/anndata/experimental/multi_files/_anncollection.py:210: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arrs.append(arr[idx])
/home/icb/kemal.inecik/tools/apps/mamba/envs/cpa/lib/python3.9/site-packages/anndata/experimental/multi_files/_anncollection.py:210: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  arrs.append(arr[idx])
/home/icb/kemal.inecik/tools/apps/mamba/envs/cpa/lib/python3.9/site-packages/anndata/experimental/multi_files/_anncollection.py:210: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future v

Defaulting to printing training loss every 100 epochs. If you want a different number, set it for the train function!
Starting training of model:


RuntimeError: mat1 and mat2 must have the same dtype